In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [4]:
import pandas as pd
import numpy as np
import sys, os
import h5py
import logging
from datetime import datetime

In [5]:
import h5py
import numpy as np
import os, sys
import subprocess
import glob
from statsmodels import robust

In [6]:
pip show ont_fast5_api #make sure the ont_fast5_api is the latest version

Name: ont-fast5-api
Version: 4.0.0
Summary: Oxford Nanopore Technologies fast5 API software
Home-page: https://github.com/nanoporetech/ont_fast5_api
Author: Oxford Nanopore Technologies, Limited
Author-email: 
License: MPL 2.0
Location: /pod/2/li-lab/Ziwei/Anaconda3/envs/nanomodel/lib/python3.6/site-packages
Requires: progressbar33, numpy, packaging, h5py
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
sys.version

'3.6.13 | packaged by conda-forge | (default, Sep 23 2021, 07:56:31) \n[GCC 9.4.0]'

# Pre-precess the data

The fast5 file is copied from /fastscratch/c-panz/2021-09-30/guppy5.0.14_supmodel_tombo/T4LambdaTF1/0/workspace

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/

00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5

000006ea-dddb-429c-8277-a81ce50da6a0.fast5

The resquiggled signal is saved under:

correctedGroup="RawGenomeCorrected_001"

basecallGroup="Basecall_1D_001"

# Define fast5 class with function features

In [1]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
#    fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list
fast5_fp = get_fast5_file_list(input_path,recursive=True)
for fast5_fn in fast5_fp:
    print(fast5_fn)

/pod/2/li-lab/Ziwei/Nanopore/daily/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5
/pod/2/li-lab/Ziwei/Nanopore/daily/test/00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5


In [15]:
import h5py
import numpy as np
import os, sys
import argparse
from statsmodels import robust
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list
    
class raw_fast5:
    def __init__(self, path, corr_group, basecall_group, basecall_subgroup, signal_group):
        self._fast5_id = os.path.basename(path)
        self._obj = h5py.File(path, mode="r")
        self._corr_group = corr_group
        self._basecall_group = basecall_group
        self._basecall_subgroup = basecall_subgroup
        self._signal_group = signal_group
        
    def __eq__(self, other):
        self._fast5_id == other._fast5_id
    
    def fast5_readid(self):
        f5 = self._obj
        try:
            read_num = list(f5[f'/{signal_group}'])[0]
        except Exception:
            raise RuntimeError('Signal not found.')
                
        read = f5[f'/{signal_group}/{read_num}']
        read_attr = dict(list(read.attrs.items()))
        read_id = read_attr['read_id'].decode('UTF-8') 
        return read_id
    
    def fast5_signal(self):
        f5 = self._obj
        try:
            read_num = list(f5[f'/{signal_group}'])[0]
        except Exception:
            raise RuntimeError('Signal not found.')
            
        signal = f5[f'/{signal_group}/{read_num}/Signal']
        signal_attr = np.array(signal[:])
        return signal_attr
    
    def fast5_event(self):
        try:
            # Get event data after tombo-requirrel
            event = self._obj[f'/Analyses/{self._corr_group}/{self._basecall_subgroup}/Events']
        except Exception:
            raise RuntimeError('Events not found.')
            
        corr_attr = dict(list(event.attrs.items()))
        
        #get read location relatively to the reference genome
        read_start_rel_to_raw = corr_attr['read_start_rel_to_raw']   
        
        #Calculate the start position relatively to the reference genome
        start = list(map(lambda x: x + read_start_rel_to_raw, event['start']))
        #Get the event length for each base
        length = event['length'].astype(np.int)
        #Get the seq information
        seq = [x.decode("UTF-8") for x in event['base']]
        
        assert len(event) == len(start) == len(seq) == len(length)
        event_attr = list(zip(start, length, seq))
        return event_attr
    
    def fast5_align(self):
        try:
            strand_align = f5[f'/Analyses/{self._corr_group}/{self._basecall_subgroup}/Alignment']
        except Exception:
            return -1
        
        read_strand = 't' if self._basecall_subgroup=='BaseCalled_template' else 'c'
        
        align_attr = dict(list(strand_align.attrs.items()))
        #Extract chrom, chrom_start, strand for alignment information
        chrom = align_attr['mapped_chrom']
        strand = align_attr['mapped_strand']
        start = align_attr['mapped_start']
        return chrom, strand, start, read_strand

In [22]:
signal_group='Raw/Reads'
corr_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5'

f5 = raw_fast5(input_path, corr_group, basecall_group, basecall_subgroup, signal_group)
event = f5.fast5_event()
read_id = f5.fast5_readid()
signal = f5.fast5_signal()
event[0:10], read_id, signal

([(1118, 3, 'T'),
  (1121, 25, 'C'),
  (1146, 3, 'C'),
  (1149, 17, 'A'),
  (1166, 14, 'G'),
  (1180, 12, 'T'),
  (1192, 3, 'C'),
  (1195, 10, 'A'),
  (1205, 11, 'T'),
  (1216, 26, 'T')],
 '000006ea-dddb-429c-8277-a81ce50da6a0',
 array([488, 460, 450, ..., 540, 538, 526], dtype=int16))

In [23]:
f5.fast5_align()

-1

In [ ]:
#channels 0pA adc: /UniqueGlobalKey/channel_id/{offset}
#digitisable range in pA: /UniqueGlobalKey/channel_id/{range}
#digitisation: /UniqueGlobalKey/channel_id/{digitisation}
#current in pA = (signal_value + channels 0pA adc) * digitisable range in pA / digitisation
import os
channel_group='UniqueGlobalKey/channel_id'

fast5_fn = os.path.join(input_path, fast5_fn)
f5 = h5py.File(fast5_fn, mode="r")

channel = f5[f'/{channel_group}']
channel_attr = dict(list(channel.attrs.items()))
current = np.array((signal + channel_attr['offset']) * channel_info['range'] / channel_info['digitisation'])
current

NameError: name 'input_path' is not defined

In [ ]:
current = signal_value + channel_attr['offset'] * channel_info['digitisation']
digi = channel_info['digitisation']
parange = channel_info['range']
offset = channel_info
scaling = parange / digi
offset, scaling


# Extract features
69 features in total

1. Nucleotide from NNNNNNNCGNNNNNNN(16 features * 4 while making binary labels) {A: 3; C: 2; G: 1; T: 0}
2. Normalized mean signal level for each base (16 features)
3. Normalized signal standard deviation for each base(16 features)
4. Length of this event in raw signal values for each base (16 features)
5. GC content in windows: 10, 20, 50, 75, 100bp (5 features)